#### Creating Asset Dataframe

In [0]:
df = spark.sql("SELECT * FROM data")

In [0]:
# Look for array field where field contains any items
asset = df.filter(size(col("payload_release.assets")) > 0).select("payload_release.*").distinct()

In [0]:
# Extracting nested arrays
asset1 = (
    asset.select(explode("assets"))
    .select("col.*")
    .withColumn("uploader_id", col("uploader.id"))
    .drop("uploader")
)

In [0]:
assetDF = (
    asset1.select(
        "id",
        "browser_download_url",
        "content_type",
        col("created_at").cast(TimestampType()).alias("created_at"),
        "download_count",
        "label",
        "name",
        "node_id",
        "size",
        "state",
        col("updated_at").cast(TimestampType()),
        "uploader_id",
        "url",
    )
    .distinct()
    .orderBy("id")
)

#### Write to database

In [0]:
assetDF.repartition(1).write.mode("overwrite").parquet("abfss://team1-project2@20230821desa.dfs.core.windows.net/SilverLayer/asset")